In [ ]:
import os
import sys
sys.path.append('..')
sys.path.append('../utils/')
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller
import numpy as np
import hiplot as hip 
#import polars as pl
from scipy.signal import welch,  butter, filtfilt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from plotly.subplots import make_subplots
from utils.EDA import*
#import endaq.plot.dashboards

In [ ]:
plt.style.use('default')

plt.rcParams.update({
    'font.size': 20,
    'axes.linewidth': 2,
    'axes.titlesize': 20,
    'axes.edgecolor': 'black',
    'axes.labelsize': 18,
    'axes.grid': True,
    'lines.linewidth': 1.5,
    'lines.markersize': 6,
    'figure.figsize': (20, 8),
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'font.family': 'Times New Roman',
    'legend.fontsize': 13,
    'legend.framealpha': 0.8,
    'legend.edgecolor': 'black',
    'legend.shadow': False,
    'legend.fancybox': True,
    'legend.frameon': True,
})

In [ ]:
path_to_dataset = "../1.ETL/Datasets/Processed/ETL2_train_label.parquet"
df = pd.read_parquet(path_to_dataset)

In [ ]:
df2 = df[(df['Process']=='OP06')]

In [ ]:
del df

In [ ]:
df1 = df2[(df2['Machine']=='M01') & (df2['Process']=='OP06')]
df1['Unique_Code'].unique()

In [ ]:
df1['Unique_Code'].unique()[16]

In [ ]:
# Filter the DataFrame for rows where the 'Label' column equals 1
df_filtered = df[df['Label'] == 1]

# Group by 'Machine' and 'Process', and extract the Unique_Code for each combination
unique_codes_per_machine_process = {}
for (machine, process), group in df_filtered.groupby(['Machine', 'Process']):
    unique_codes = group['Unique_Code'].unique()
    if machine not in unique_codes_per_machine_process:
        unique_codes_per_machine_process[machine] = {}
    unique_codes_per_machine_process[machine][process] = unique_codes

# Print the Unique Codes associated with Label 1 for each machine and process combination
for machine, processes in unique_codes_per_machine_process.items():
    print(f"Machine {machine}:\n")
    for process, codes in processes.items():
        print(f"  Process {process} with anomaly: {codes}\n")

In [ ]:
df1 = df[(df['Machine']=='M01') & (df['Process']=='OP06')]

In [ ]:
del df

In [ ]:
#label_df = pd.read_parquet('../1.ETL/Datasets/Processed/ETL2_train_label.parquet')
#label_df[(label_df['Label']==1)]

In [ ]:
#df = label_df

# Understanding the data

- Data acquisition is done indirectly using Bosch CISS accelerometer sensors mounted at the spindle housing's rear end;
- The sensors maintain a constant distance to the tool center point, and their axes align with the machine's linear motion axi;
- The frequencies of the tool operations are in a range of 75 Hz to 1 kHz;

- The machine performs a sequence of several operations using different tools on aluminium parts to work the specified design;
- The machines produce different parts and the process flow changes over time. To study the drift between machines and over time, the dataset is built with 15 different tool operations that run on all 3 machines at different time frames.

- Table gives an overview on the characteristics of the different operations:


In [ ]:
plot_by_code_index_matplotlib(df, process='OP06', machine = 'M01', axis='Z_axis', decimation_factor=10)

In [ ]:
plot_by_code_index_matplotlib(df1, process='OP06', machine = 'M01',axis='Y_axis', decimation_factor=10)

In [ ]:
plot_by_code_index_matplotlib(df1, process='OP06', machine = 'M01',axis='X_axis', decimation_factor=10)

In [ ]:
plot_all_axis_matplotlib(df, process='OP06', machine='M01', by_code=False,decimation_factor=100)

In [ ]:
df2.columns

### Data frequency for diffent process

In [ ]:
# Função para calcular a FFT
def calculate_fft(df, column_name, unique_code):
    # Filtrando os dados por 'Unique_Code'
    data_filtered = df[df['Unique_Code'] == unique_code]
    freq = 2000
    
    fft_values = np.fft.fft(data_filtered[column_name])
    n = len(fft_values)
    frequencies = np.fft.fftfreq(n, d=1/freq)
    magnitudes = np.abs(fft_values)[:n // 2][1:]  # removendo a componente dc do sinal
    return frequencies[:n // 2][1:], magnitudes

In [ ]:
df2.columns

In [ ]:
df2[df2['Unique_Code']=='M01_OP06_Aug_2021_4']

In [ ]:
fig = go.Figure()

#Unique Codes 
unique_codes = ['M01_OP06_Aug_2019_1','M01_OP06_Aug_2019_3','M01_OP06_Aug_2021_2','M01_OP06_Aug_2021_4', 'M02_OP06_Aug_2019_1', 'M03_OP06_Aug_2019_1']

for code in unique_codes:
    frequencies, magnitudes = calculate_fft(df2, 'Y_axis', code)

    fig.add_trace(go.Scatter(x=frequencies, y=magnitudes, mode='lines', name=code))

fig.update_layout(
    title='FFT - Y_axis',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Magnitude',
    autosize=False,    
    width=1200)

fig.show()


It doesn't seem like there are significant differences between operations that might indicate the presence of an anomaly operation, but there does appear to be a change in the natural frequency between the machines.

In [ ]:
fig = go.Figure()

# Loop over each Unique_Code to add histograms to the same plot
for code in df1['Unique_Code'].unique():
    # Filter data by Unique_Code and take a sample of it
    sample_data = df1[df1['Unique_Code'] == code].sample(frac=0.3, random_state=42)

    # Add histogram to the chart
    fig.add_trace(go.Histogram(
        x=sample_data['X_axis'],
        name=f'{code}',  
        opacity=0.75,  # Adjust opacity for better visualization of overlaps
        histnorm='probability'  # Normalize histogram to show proportions
    ))

# Update layout for better visualization and comparison
fig.update_layout(
    title_text='X_axis distribution - M01 OP06', 
    xaxis_title_text='X_axis',
    yaxis_title_text='Count', 
    bargap=0.1, 
    barmode='overlay')

fig.show()

In [ ]:
fig = go.Figure()

# Loop over each Unique_Code to add histograms to the same plot
for code in df1['Unique_Code'].unique():
    # Filter data by Unique_Code and take a sample of it
    sample_data = df1[df1['Unique_Code'] == code].sample(frac=0.3, random_state=42)

    # Add histogram to the chart
    fig.add_trace(go.Histogram(
        x=sample_data['Y_axis'],
        name=f'{code}',  
        opacity=0.75,  # Adjust opacity for better visualization of overlaps
        histnorm='probability'  # Normalize histogram to show proportions
    ))

# Update layout for better visualization and comparison
fig.update_layout(
    title_text='Y_axis distribution - M01 OP06', 
    xaxis_title_text='Y_axis', 
    yaxis_title_text='Count', 
    bargap=0.1,
    barmode='overlay')

fig.show()

In [ ]:
fig = go.Figure()

# Loop over each Unique_Code to add histograms to the same plot
for code in df1['Unique_Code'].unique():
    # Filter data by Unique_Code and take a sample of it
    sample_data = df1[df1['Unique_Code'] == code].sample(frac=0.3, random_state=42)

    # Add histogram to the chart
    fig.add_trace(go.Histogram(
        x=sample_data['Z_axis'],
        name=f'{code}',  
        opacity=0.75,  # Adjust opacity for better visualization of overlaps
        histnorm='probability'  # Normalize histogram to show proportions
    ))

# Update layout for better visualization and comparison
fig.update_layout(
    title_text='Z_axis distribution - M01 OP06', 
    xaxis_title_text='Z_axis', 
    yaxis_title_text='Count', 
    bargap=0.1,
    barmode='overlay')

fig.show()

# Filtering the signal

In [ ]:
fs = 2000  # Sampling rate in Hz

# Desired columns
columns = ['X_axis', 'Y_axis', 'Z_axis']

code = df1['Unique_Code'].unique()[0]
selected_df = df1[(df1['Unique_Code'] == code)]

# Number of rows and columns for the subplots
num_rows = 3
num_columns = 1

# Creating the subplots
plt.figure(figsize=(10, 8))  # Adjust the figure size as needed

for i, column in enumerate(selected_df[columns].columns):
    plt.subplot(num_rows, num_columns, i + 1)
    
    # Calculate the FFT using the Welch method with the specified sampling rate
    f, Pxx = welch(selected_df[column], fs=fs)
    
    plt.plot(f, np.sqrt(Pxx), label=column)
    plt.xlabel('Frequency [Hz]', fontsize=18)
    plt.ylabel(f'{column}', fontsize=18)
    plt.title(f'FFT of {column}', fontsize=20)
    plt.tick_params(axis='x', which='major', labelsize=16)
    plt.tick_params(axis='y', which='major', labelsize=16)
    plt.tight_layout()
    plt.grid()

plt.show()


Apparently, this signal cannot be filtered because it contains important components at higher frequencies—it's not just noise.

In [ ]:
nahlkdlkanl,ndlçnlçkdsnjçlk

In [ ]:
fs = 2000  # Sample frequency (Hz)
cutoff = 500  # Cut-off frequency
order = 4  # Filter order

def filter_data(df, colunas, fs, cutoff_frequency, order):
    """
    Aplica um filtro Butterworth passa-baixo às colunas especificadas, separadamente para cada viagem, 
    e adiciona os dados filtrados ao DataFrame original como novas colunas.

    Args:
    - df: DataFrame original.
    - colunas: Lista de colunas a serem filtradas.
    - fs: Taxa de amostragem em Hz.
    - cutoff_frequency: Frequência de corte em Hz.
    """
    # Projetar o filtro Butterworth passa-baixo
    nyquist = 0.5 * fs
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)

    # Iterar sobre cada viagem
    for code in df['Unique_Code'].unique():
        # Selecionar dados da viagem atual
        code_indices = df['Unique_Code'] == code

        # Filtrar cada coluna selecionada para a viagem atual
        for coluna in colunas:
            coluna_filtrada = f'{coluna}_filt'
            df.loc[code_indices, coluna_filtrada] = filtfilt(b, a, df.loc[code_indices, coluna])


In [ ]:
colunas_para_filtrar = ['X_axis','Y_axis','Z_axis']
filter_data(df1, colunas_para_filtrar, fs = 2000, cutoff_frequency = 500, order = 4)

In [ ]:
def plot_filtered_data(df, code, colunas):
    """
    Plota a comparação entre dados originais e filtrados para uma viagem e intervalo de Km específicos.

    Args:
    - df: DataFrame contendo dados originais e filtrados.
    - trip: Nome da viagem (ex: 'Trip 1').
    - colunas: Lista de colunas a serem plotadas.
    - km_inicial: Início do intervalo de quilometragem para plotar.
    - km_final: Fim do intervalo de quilometragem para plotar.
    """
    code = df['Unique_Code'].unique()[code]
    # Filtrar o DataFrame pela viagem e intervalo de Km
    df_filtered = df[(df['Unique_Code'] == code)]

    # Plotando os dados
    for coluna in colunas:
        plt.figure(figsize=(18, 6))
        plt.plot(df_filtered['Time'], df_filtered[coluna], label=f'Original {coluna}', color='blue')
        plt.plot(df_filtered['Time'], df_filtered[f'{coluna}_filt'], label=f'Filtrado {coluna}', color='red')
        plt.title(f'Compare Original x Filtered - {coluna} - {code}')
        plt.xlabel('Time')
        plt.ylabel(coluna)
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
colunas_para_plotar = ['X_axis','Y_axis','Z_axis']
plot_filtered_data(df1, 16, colunas_para_plotar)

In [ ]:
colunas_para_plotar = ['X_axis','Y_axis','Z_axis']
plot_filtered_data(df1, 0, colunas_para_plotar)

In [ ]:
colunas_para_plotar = ['X_axis','Y_axis','Z_axis']
plot_filtered_data(df1, 18, colunas_para_plotar)

# Time series analysis

The ACF measures the linear relationship between a time series and its lagged values. It calculates the correlation coefficient between the series and its lagged values at different time lags. 

The ACF plot can provide answers to the following questions:

1. Is the observed time series white noise/random?
2. Is an observation related to an adjacent observation, an observation twice-removed, and so on?
3. Can the observed time series be modeled with an MA model? If yes, what is the order?

Source: https://www.kaggle.com/code/iamleonie/time-series-interpreting-acf-and-pacf

On the other hand, the PACF measures the correlation between a time series and its lagged values while removing the effects of intermediate lags. It represents the correlation between two variables after removing the influence of other variables in between. By analyzing the PACF plot, you can identify the direct influence of each lag on the current value of the time series, independent of the other lags.

The PACF plot can provide answers to the following question:

1. Can the observed time series be modeled with an AR model? If yes, what is the order?

In [ ]:
df1['Unique_Code'].unique()

In [ ]:
df1[df1['Unique_Code'] == 'M01_OP06_Aug_2019_5']['X_axis']

In [ ]:
def check_stationarity(series):
    # Copied from https://machinelearningmastery.com/time-series-data-stationary-python/

    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

In [ ]:
check_stationarity(df1[df1['Unique_Code'] == 'M01_OP06_Feb_2019_1']['X_axis'])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20, 6))
ax=ax.ravel()

# Plot ACF
plot_acf(df1[df1['Unique_Code'] == 'M01_OP06_Feb_2019_1']['X_axis'], ax=ax[0], lags=500)
ax[0].set_xlabel('Lags')
ax[0].set_ylabel('ACF')
ax[0].set_title('Autocorrelation Function (ACF)')
ax[0].set_xlim([0,100])

# Plot PACF
plot_pacf(df1[df1['Unique_Code'] == 'M01_OP06_Feb_2019_1']['X_axis'], ax=ax[1], lags=500)
ax[1].set_xlabel('Lags')
ax[1].set_ylabel('PACF')
ax[1].set_title('Partial Autocorrelation Function (PACF)')
ax[1].set_xlim([0,40])

plt.suptitle('M01_OP06_Feb_2019_1')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20, 6))
ax=ax.ravel()

# Plot ACF
plot_acf(df1[df1['Unique_Code'] == 'M01_OP06_Aug_2021_3']['X_axis'], ax=ax[0], lags=500)
ax[0].set_xlabel('Lags')
ax[0].set_ylabel('ACF')
ax[0].set_title('Autocorrelation Function (ACF)')
ax[0].set_xlim([0,100])

# Plot PACF
plot_pacf(df1[df1['Unique_Code'] == 'M01_OP06_Aug_2021_3']['X_axis'], ax=ax[1], lags=500)
ax[1].set_xlabel('Lags')
ax[1].set_ylabel('PACF')
ax[1].set_title('Partial Autocorrelation Function (PACF)')
ax[1].set_xlim([0,40])

plt.suptitle('M01_OP06_Aug_2021_3')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20, 6))
ax=ax.ravel()

# Plot ACF
plot_acf(df1[df1['Unique_Code'] == 'M01_OP06_Aug_2019_5']['X_axis'], ax=ax[0], lags=500)
ax[0].set_xlabel('Lags')
ax[0].set_ylabel('ACF')
ax[0].set_title('Autocorrelation Function (ACF)')
ax[0].set_xlim([0,100])

# Plot PACF
plot_pacf(df1[df1['Unique_Code'] == 'M01_OP06_Aug_2019_5']['X_axis'], ax=ax[1], lags=500)
ax[1].set_xlabel('Lags')
ax[1].set_ylabel('PACF')
ax[1].set_title('Partial Autocorrelation Function (PACF)')
ax[1].set_xlim([0,40])

plt.suptitle('M01_OP06_Aug_2019_5')
plt.tight_layout()
plt.show()

## Time series decomposition

A time series is composed by systematic and unsystematic components.

- Systematic: Components of the time series that have consistency or recurrence and can be described and modeled.
- Non-Systematic: Components of the time series that cannot be directly modeled.

A given time series is thought to consist of three systematic components including level, trend, seasonality, and one non-systematic component called noise. These components are defined as follows:

- Level: The average value in the series.
- Trend: The increasing or decreasing value in the series.
- Seasonality: The repeating short-term cycle in the series.
- Noise: The random variation in the series.

Reference: https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/

In [ ]:
df1

In [ ]:
df1.reset_index(drop=True, inplace=True)

# Definir o número de amostras por ciclo
samples_per_cycle = 90 * 2000  # 90 segundos * 2000 amostras/segundo

# Calcular a tendência usando média móvel com janela de 2000 pontos (1 segundo)
df1['trend'] = df1['X_axis'].rolling(window=2000, center=True).mean()
df1['trend'].fillna(method='bfill', inplace=True)  # Preenche NaN no início do DataFrame
df1['trend'].fillna(method='ffill', inplace=True)  # Preenche NaN no final do DataFrame

# Detrend a série
df1['detrended'] = df1['X_axis'] - df1['trend']

# Calcular a componente sazonal, assumindo que cada ciclo é aproximadamente de 90 segundos
# Identificar o ciclo baseado no índice
df1['cycle'] = df1.index // samples_per_cycle
df1['seasonality'] = df1.groupby('cycle')['detrended'].transform('mean')

# Calcular os resíduos
df1['resid'] = df1['detrended'] - df1['seasonality']

df.dropna(inplace=True)


In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 8), sharex=True)

axes[0].plot(df1['X_axis'], label='Original')
axes[0].set_title('Original Data')
axes[0].legend()

axes[1].plot(df1['trend'], label='Trend', color='orange')
axes[1].set_title('Trend')
axes[1].legend()

axes[2].plot(df1['seasonality'], label='Seasonality', color='green')
axes[2].set_title('Seasonality')
axes[2].legend()

axes[3].plot(df1['resid'], label='Residuals', color='red')
axes[3].set_title('Residuals')
axes[3].legend()


plt.tight_layout()
plt.show()


In [ ]:
season_df

In [ ]:
season_df

In [ ]:
def plot_correlation(df, machine, process, unique_code=None):
    """
    Plota a matriz de correlação dos eixos X, Y e Z para uma máquina e processo específicos,
    com a opção de filtrar por um código único e sem exibir o triângulo superior da matriz.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame que contém os dados.
    - machine (str): A máquina a ser filtrada.
    - process (str): O processo a ser filtrado.
    - unique_code (str, opcional): O código único a ser filtrado. Se None, considera todos os códigos.
    """
    # Filtrar por Machine, Process, e opcionalmente por Unique_Code
    if unique_code:
        filtered_df = df[(df['Machine'] == machine) & (df['Process'] == process) & (df['Unique_Code'] == unique_code)]
    else:
        filtered_df = df[(df['Machine'] == machine) & (df['Process'] == process)]

    if filtered_df.empty:
        print("Não foram encontrados dados para os filtros fornecidos.")
        return
    
    # Calcular a matriz de correlação
    correlation_matrix = filtered_df[['X_axis', 'Y_axis', 'Z_axis']].corr()

    # Criar a máscara para ocultar o triângulo superior
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)

    # Plotar o heatmap com a máscara
    plt.figure(figsize=(6,5))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', fmt=".2f", cbar_kws={'shrink': .8})
    plt.title(f'Machine: {machine}, Process: {process}' + (f',\n Unique_Code: {unique_code}' if unique_code else ''))
    plt.grid(True, which='both', color='grey', linestyle='-', linewidth=0.5)
    plt.show()


In [ ]:
plot_correlation(df, machine = 'M01', process = 'OP06')

In [ ]:
plot_correlation(df, machine = 'M01', process = 'OP06', unique_code= 'M01_OP06_Aug_2019_5')

In [ ]:
plot_correlation(df, machine = 'M01', process = 'OP06', unique_code= 'M01_OP06_Aug_2019_2')

The correlation between sensors is very low and apparently increases only during operations with anomalies.

## Mainfold learning to visualize the data

In [ ]:
df1 = df[(df['Machine']=='M01') & (df['Process']=='OP06')]

In [ ]:
df1['Label'] = df1['Label'].astype('str')

In [ ]:
fig = px.scatter(df1[::100], x='Z_axis', y='Y_axis', color='Label',
                 labels={
                     'Z_axis': 'Z Axis',
                     'Y_axis': 'Y Axis',
                     'Label': 'Label'
                 },
                 title='Z and Y axis for M01 - OP06')

fig.update_layout(width=800, height=600)

fig.show()


In [ ]:
fig = px.scatter(df1[::100], x='Z_axis', y='X_axis', color='Label',
                 labels={
                     'Z_axis': 'Z Axis',
                     'Y_axis': 'X Axis',
                     'Label': 'Label'
                 },
                 title='Z and X axis for M01 - OP06')

fig.update_layout(width=800, height=600)

fig.show()

In [ ]:
fig = px.scatter(df1[::100], x='Y_axis', y='X_axis', color='Label',
                 labels={
                     'Z_axis': 'Y Axis',
                     'Y_axis': 'X Axis',
                     'Label': 'Label'
                 },
                 title='Y and X axis for M01 - OP06')

fig.update_layout(width=800, height=600)

fig.show()

In [ ]:
sample_main_df = df1.sample(frac=0.1, random_state=0)

In [ ]:
sample_main_df

In [ ]:
fig = px.scatter_3d(sample_main_df, x='X_axis', y='Y_axis', z='Z_axis',
              color='Label')
fig.show()

In [ ]:
features = ['X_axis','Y_axis','Z_axis']
X = sample_main_df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns=features)

In [ ]:
pca = PCA(n_components=2) 
X_pca = pca.fit_transform(X_scaled)

explained_var = pca.explained_variance_ratio_
print("Explained variance for each component:", explained_var)

In [ ]:
pca_df = pd.DataFrame(data = X_pca[:, :2], columns = ['PC1', 'PC2'])
pca_df['Label'] = df1['Label'].values 

pca_df

In [ ]:
fig = px.scatter(pca_df[::100], x='PC1', y='PC2', color='Label',
                 labels={
                     'PC1': 'PC1',
                     'PC2': 'PC1'
                 },
                 title='PCA')
fig.show()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
tsne_results = tsne.fit_transform(X_scaled)

# Criando um novo DataFrame para os resultados do t-SNE
df_tsne = pd.DataFrame(data=tsne_results, columns=['TSNE1', 'TSNE2'])